In [1]:
from PIL import Image
import numpy as np
from flask import Flask, request, jsonify
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
app = Flask(__name__)
model = tf.keras.models.load_model("model")
normal_vector = np.load("normal_vector.npy")
input_shape = (112, 112)

In [4]:
def prepare_img(img_file):
    img = Image.open(img_file.stream)
    img = np.array(img)
    img = np.expand_dims(img, axis=2)
    img = tf.image.resize(img, input_shape)
    img = preprocess_input(img)
    return img.numpy()
    

In [5]:
def predict_class(test_data, normal_template_vector, therashold):
    """
    test_data: each row represent feature vector of a photo
    
    """
    sim = np.dot(test_data, normal_template_vector)
    return (sim < therashold).astype(int) # true for an anomaly 

In [6]:
@app.route('/predict',methods=['POST'])
def predict():
    '''
    For direct API calls trought request
    '''
    # read and preprocess images
    imgs = request.files
    imgs_lst = imgs.to_dict(flat=False)['image']
    
    processed_imgs = []
    for img in imgs_lst:
        processed_imgs.append(prepare_img(img))

    processed_imgs = np.array(processed_imgs)
    
    # get the l2 normalized 512- dimensional feature representations of the images. 
    feature_vectors = model.predict(processed_imgs)
    
    #predict
    prediction = predict_class(feature_vectors, normal_vector, 0.81)
    
    return jsonify({'result': prediction.tolist()})


In [ ]:
if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Dec/2021 22:23:47] "POST /predict HTTP/1.1" 200 -
